In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn import datasets

In [2]:
# preparing the dataset into inputs (feature matrix) and outputs (target vector)
data = datasets.load_boston()  # fetch the data
X = data.data  # feature matrix
y = data.target  # target vector

# split the data into training and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
X_test.shape

(152, 13)

In [6]:
def draw_bootstrap_sample(rng, X, y):
    sample_indices = np.arange(X.shape[0])
    bootstrap_indices = rng.choice(
        sample_indices, size=sample_indices.shape[0], replace=True
    )
    return X[bootstrap_indices], y[bootstrap_indices]

def bias_variance_decomp(estimator, X_train, y_train, X_test, y_test, num_rounds=200, random_seed=20):
    rng = np.random.RandomState(random_seed)

    all_pred = []
    for i in range(num_rounds):
        # do bootstrap sampling, i.e., sampling with replacement
        X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)

        # fit a model on bootstrap samples and make prediction on test samples
        pred = estimator.fit(X_boot, y_boot).predict(X_test)
        all_pred.append(pred)
        
    all_pred = np.array(all_pred)

    # calculate MSE
    avg_mse = ((all_pred - y_test[None,:])**2).mean() # y_test[None,:] will reshape y_test from (N,) to (1,N)

    # average prediction of all bootstrap models on test set
    avg_predictions = np.mean(all_pred, axis=0)
    
    # calculate bias squared
    avg_bias = np.sum((avg_predictions - y_test) ** 2) / y_test.size
    
    # calculate variance
    avg_var = np.sum((avg_predictions - all_pred) ** 2) / all_pred.size

    return avg_mse, avg_bias, avg_var

In [11]:
# define the model
model = LinearRegression()
 
# estimating the bias and variance
avg_mse, avg_bias, avg_var = bias_variance_decomp(model, X_train,
                                                            y_train, X_test,
                                                            y_test,
                                                            num_rounds=500,
                                                            random_seed=0)

# summary of the results
print('Average mean-squared error: %.3f' % avg_mse)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)


Average mean-squared error: 19.758
Average bias: 18.544
Average variance: 1.214


In [8]:
model.coef_

array([ 6.67138004e-02,  5.31815368e-02,  3.73125040e-02,  2.63243995e+00,
       -1.84812754e+01,  3.41149785e+00, -1.53768608e-02, -1.60135755e+00,
        2.05252340e-01, -1.13375173e-02, -7.07021065e-01,  7.35349784e-03,
       -5.78397304e-01])

In [16]:
# define the model
model = Ridge(alpha=10)

# estimating the bias and variance
avg_mse, avg_bias, avg_var = bias_variance_decomp(model, X_train,
                                                            y_train, X_test,
                                                            y_test,
                                                            num_rounds=500,
                                                            random_seed=0)

# summary of the results
print('Average mean-squared error: %.3f' % avg_mse)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
model.coef_

Average mean-squared error: 19.813
Average bias: 18.717
Average variance: 1.096


array([ 0.08015184,  0.05776007, -0.02968158,  1.361545  , -1.8369329 ,
        3.26361659, -0.03101414, -1.40953077,  0.2482763 , -0.01678675,
       -0.47004686,  0.00965353, -0.61751086])